

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_TR.ipynb)

# **Detect entities in Turkish text**

## 1. Colab Setup, loading necassary libraries

In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start the Spark session

In [3]:
spark = sparknlp.start()

In [4]:
spark.version

'3.1.2'

## 3. Select the DL model

In [5]:
# If you change the model, re-run all the cells below.
# Applicable embedding models: "glove_840B_300" and "bert_multi_cased"

# MODEL_NAME = 'turkish_ner_840B_300'
MODEL_NAME = 'turkish_ner_bert'

## 4. Some sample examples

In [6]:
# Enter examples to be transformed as strings in this list

text_list = ["""William Henry Gates III (28 Ekim 1955 doğumlu), Amerikalı bir iş adamı, \
yazılım geliştirici, yatırımcı ve hayırseverdir. En çok Microsoft şirketinin kurucu \
ortağı olarak bilinir. William Gates , Microsoft şirketindeki kariyeri boyunca başkan, \
icra kurulu başkanı, başkan ve yazılım mimarisi başkanı pozisyonlarında bulunmuş, \
aynı zamanda Mayıs 2014'e kadar en büyük bireysel hissedar olmuştur. O, 1970'lerin \
ve 1980'lerin mikrobilgisayar devriminin en tanınmış girişimcilerinden ve öncülerinden biridir. \
Seattle Washington'da doğup büyüyen William Gates, 1975'te New Mexico Albuquerque'de \
çocukluk arkadaşı Paul Allen ile Microsoft şirketini kurdu; dünyanın en büyük kişisel \
bilgisayar yazılım şirketi haline geldi. William Gates, Ocak 2000'de icra kurulu başkanı \
olarak istifa edene kadar şirketi başkan ve icra kurulu başkanı olarak yönetti ve \
daha sonra yazılım mimarisi başkanı oldu. 1990'ların sonlarında, William Gates rekabete \
aykırı olduğu düşünülen iş taktikleri nedeniyle eleştirilmişti. Bu görüş, çok sayıda \
mahkeme kararıyla onaylanmıştır. Haziran 2006'da William Gates, Microsoft şirketinde \
yarı zamanlı bir göreve ve 2000 yılında eşi Melinda Gates ile birlikte kurdukları özel \
hayır kurumu olan B&Melinda G. Vakfı'nda tam zamanlı çalışmaya geçeceğini duyurdu. \
Görevlerini kademeli olarak Ray Ozzie ve Craig Mundie'ye devretti. Şubat 2014'te \
Microsoft başkanlığından ayrıldı ve yeni atanan icra kurulu başkanı, Satya Nadella'yı \
desteklemek için teknoloji danışmanı olarak yeni bir göreve başladı.""",
    
"Mona Lisa, Leonardo tarafından yaratılan 16. yüzyıldan kalma bir yağlı boya tablodur. \
Tablo, Paris'teki Louvre Müzesi'nde sergileniyor.",
             
"""Facebook, 4 Şubat 2004 tarihinde TheFacebook adıyla başlatılan bir sosyal ağ hizmetidir. \
Mark Zuckerberg tarafından üniversite oda arkadaşları ve Harvard Üniversitesi öğrencileri \
Eduardo Saverin, Andrew McCollum, Dustin Moskovitz ve Chris Hughes ile birlikte kurulmuştur. \
Web sitesinin üyeliği başlangıçta kurucular tarafından Harvard öğrencileriyle sınırlıydı, ancak \
Boston bölgesindeki diğer kolejlere, Ivy-Ligine ve kademeli olarak ABD ve Kanada'daki çoğu üniversiteye genişletildi.""",
            
"""Geoffrey Everest Hinton, çoğu yapay sinir ağları üzerine yaptığı çalışmalarla tanınan \
İngiliz Kanadalı bir bilişsel psikolog ve bilgisayar bilimcisidir. 2013'ten beri zamanını \
Google ve Toronto Üniversitesi için ikiye ayırıyor. 2017 yılında, Toronto'da bulunan \
V. Enstitüsü'nün kurucu ortağı ve Bilimsel Konular Başdanışmanı oldu.""",
             
"""John Snow'a, Alaska'ya taşınmak istediğimi söylediğimde, orada bir Starbucks \
Cafe firması bulmakta zorlanacağım konusunda beni uyardı."""
            ]

## 5. Define Spark NLP pipeline

In [7]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

embeddings = None
public_ner = None

if MODEL_NAME == 'turkish_ner_840B_300' :
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', "xx").\
                    setInputCols(["sentence", 'token']).\
                    setOutputCol("embeddings").\
                    setCaseSensitive(True)

    public_ner = NerDLModel.pretrained('turkish_ner_840B_300', 'tr') \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")
    
elif MODEL_NAME == 'turkish_ner_bert' :
    embeddings = BertEmbeddings.pretrained('bert_multi_cased', 'xx') \
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    public_ner = NerDLModel.pretrained('turkish_ner_bert', 'tr') \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["sentence", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

nlp_pipeline = Pipeline(stages=[ documentAssembler, sentenceDetector,
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

bert_multi_cased download started this may take some time.
Approximate size to download 638.6 MB
[OK!]
turkish_ner_bert download started this may take some time.
Approximate size to download 15.5 MB
[OK!]


## 6. Run the pipeline

In [8]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

In [9]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)